In [1]:
import importlib
import misc; importlib.reload(misc)
import ffmpeg_wrap; importlib.reload(ffmpeg_wrap)
import signal_tools as st; importlib.reload(st)
import fft; importlib.reload(fft) 

import pylab as plt 
import numpy as np
import warnings
import shutil
from os import path
from collections import defaultdict, OrderedDict

ImportError: cannot import name 'Callable' from 'collections' (F:\Anaconda\Lib\collections\__init__.py)

In [ ]:
import os
import sys
curdrive = '/media/simon/Die is Simon se eie hardeskyf' #path.abspath(os.getcwd())[:2]

japdir = curdrive+"/Backups/#Torrents/Heidi_Girl_of_the_Alps_BD_Simu"
afrdir = curdrive+"/Backups/#Torrents/Heidi-VHS"

import signal_tools as st; importlib.reload(st)
warnings.filterwarnings('ignore')

filesafr = [file for file in misc.dirfiles(afrdir) if file.lower().endswith('.mp4')]
filesjap = [file for file in misc.dirfiles(japdir) if file.lower().endswith('.mkv')]

fnames_lst = [st.generate_filenames(file_afr, file_jap, file_afr, id=i+1)
              for i, (file_afr, file_jap) in enumerate(zip(filesafr, filesjap))]

In [ ]:
for ii, fnames in enumerate(fnames_lst):
    
    print("*******************************")
    print("We are busy with video: "+fnames.discard_video)
    print("*******************************")
    #################################################################
    #
    #  Part 1
    #################################################################
    print('Reading frames for '+fnames.discard_video)
    v1, frames1 = st.read_2D_video_signal(fnames.discard_video)
    print('Reading frames for '+fnames.desired_video)
    v2, frames2 = st.read_2D_video_signal(fnames.desired_video)

    np.save(fnames.discard_signal, v1)
    np.save(fnames.desired_signal, v2)
    np.save(fnames.discard_frames, frames1)
    np.save(fnames.desired_frames, frames2)
    
    print("Reading audio from "+fnames.desired_audio)
    arr, freq1 = ffmpeg_wrap.read_as_audio(fnames.desired_audio)
    np.save(fnames.input_audio, arr)
    
    #################################################################
    #
    #  Part 2
    #################################################################
    v1 = np.load(fnames.discard_signal)
    v2 = np.load(fnames.desired_signal)
    
    shutil.rmtree(fnames.dmap_dumps, ignore_errors=True)
    os.makedirs(fnames.dmap_dumps)
    
    print("Generating delay maps: ", end='', flush=True)
    mapping = []
    n_maps = 30
    for i in range(n_maps):
        if i%5 ==0:
            print("%d/%d "%(i+1, n_maps), end='', flush=True)

        #                                            (2min   , 10s ) 
        dmap = st.find_mapping(v1, v2, bordercontrol=(24*60*2, 24*10))

        np.save(fnames.dmap_dumps+'/%03d.npy'%i, dmap)

        mapping.append(dmap)
    print()

    
    #################################################################
    #
    #  Part 3
    #################################################################
    print("Find best fit from these multiple map attempts ...")
    mapping = [np.load(i) for i in misc.dirfiles(fnames.dmap_dumps)]

    medianmap = np.median(mapping, axis=0)

    np.save(fnames.dmap, medianmap)

    frames1 = np.load(fnames.discard_frames)

    dmap_fix = st.fix_dmap_warps(medianmap, fnames)

    np.save(fnames.dmap_smooth, dmap_fix)
    
    plt.figure(figsize=(12,4))
    plt.plot(dmap_fix)
    plt.plot(medianmap)
    plt.xlabel(os.path.split(fnames.desired_video)[-1])
    plt.show()

    
    #################################################################
    #
    #  Part 4
    #################################################################
    print("Warp the audio ...")
    st.warp_audio(fnames)
    
    #################################################################
    #
    #  Part 5
    #################################################################
    print("Fit the audio to the video ...")
    st.mux_audio_to_video(fnames)
    